##  Web scraping - Familydollar

**Estabelecendo uma meta para o projeto de web-scraping. - Para um projeto bem sucedido deve-se seguir os seguintes passos.**

- Estamos coletando informações que valem o esforço necessário para criar um web scraping em funcionamento.
- Estamos baixando informações que podem ser coletadas legal e eticamente por um web scraping.
- Temos algum conhecimento de como encontrar as informações de destino no código HTML.
- Temos as ferramentas certas: nesse caso, são as bibliotecas BeautifulSoup e requests.
- Sabemos (ou estamos dispostos a aprender) como analisar objetos JSON.
- Temos habilidade para manipular dados com pandas.


- O site o quel faremos web-scraping é: https://www.familydollar.com/locations/id/
        
- Estas lojas possuem uma grande presença em areas rurais nos EUA, o objetivo é descobrir quantas existem em estados mais ruralizados do sul (Lembrando que é apenas um exemplo para demonstração de conhecimento, podemos posterimormente fazer com casos da nossa realizade brasileira.)

In [1]:
# importando as bibliotecas utilizadas

import requests # para fazer as requisições em html
from bs4 import BeautifulSoup # ferramenta para analisar dados html
import json # para analisar os dados
from pandas import DataFrame as df 

O BeautifulSoup pega o conteúdo HTML ou XML e o transforma em uma complexa árvore de objetos. Aqui estão vários tipos de objetos comuns que usaremos:

- BeautifulSoup — o conteudo que será analisado.
- Tag — uma tag HTML padrão, o principal tipo de elemento bs4 que você encontrará
- NavigableString — uma string dentro das tags
- Comment - um tipo especial de NavigableString

In [16]:
# Para garantir que as requisições analisem corretamente o texto, deixaremos as variaveis abaixo não utilizando o default.

#allow_redirects = ajusta a biblioteca de solicitações para não seguir os redirecionamentos. Caso necessário.

page = requests.get("https://www.familydollar.com/locations/id/", allow_redirects=False)#indicar a URL da página
page.encoding = 'ISO-885901'  #No caso por estarmos fazendo o webscraping de um site em ingles, em latin mudar utf-8
soup = BeautifulSoup(page.text, 'html.parser')

**Determinando como extrair um conteudo relevante**

 Visualizar o código HTML da página é essencial. Há alguns modos de se fazer isso.
 
 - Você pode ver todo o codigo da fonte por meio do terminal do Python (Não recomendado).
 - Ou visualizar o codigo fonte por meio do seu navegador(clique no botão direito da pagina e selececione "Exibir codigo fonte da página") Essa é a maneira mais confiável de encontrar seu conteúdo de destino .

- Vamos utilizar a visualização por meio do navegador, nesse caso, preciso encontrar meu conteúdo de destino - um endereço, cidade, estado e CEP - nesse vasto codigo HTML. Muitas vezes, uma simples pesquisa na fonte da página (ctrl + F) gera a seção em que meu local de destino está localizado. Depois que posso realmente ver um exemplo do meu conteúdo de destino (o endereço de pelo menos uma loja), procuro um atributo ou marca que diferencia esse conteúdo do restante.
- Procurando a palavra chave State, foi possivel achar onde estão os links que indicam os locais das lojas. No caso foram achadas esse endereços entre as tags "href". Portanto vamos procurar por meio do find_all.

In [7]:
dollar_tree_list = soup.find_all('href')
dollar_tree_list

[]

- Procurar href não produziu nada. Isso pode ter falhado porque o href está aninhado dentro da lista de itens da classe. Para a próxima tentativa, pesquise item_list. Como "class" é uma palavra reservada no Python, class_ é usado em seu lugar.

In [17]:
#Procurando agora itemlist tag que contem os endereços.

dollar_tree_list = soup.find_all(class_ = 'itemlist')

In [19]:
for i in dollar_tree_list[:2]:
  print(i)

<div class="itemlist" data-fl="A"><a class="ga_w2gi_lp" data-gaact="Click_to_CityPage" data-galoc="Aberdeen - ID" dta-linktrack="City index page - Aberdeen" href="https://www.familydollar.com/locations/id/aberdeen/">Aberdeen</a></div>
<div class="itemlist" data-fl="A"><a class="ga_w2gi_lp" data-gaact="Click_to_CityPage" data-galoc="American Falls - ID" dta-linktrack="City index page - American Falls" href="https://www.familydollar.com/locations/id/american-falls/">American Falls</a></div>


In [20]:
#Agora vamos aprender mais sobre o objeto:
len(dollar_tree_list)

48

In [24]:
type(dollar_tree_list)

bs4.element.ResultSet

In [23]:
#O conteúdo deste BeautifulSoup "ResultSet" pode ser extraído usando .contents. 
#Este também é um bom momento para criar um único exemplo representativo.

example = dollar_tree_list[20] # a representative example
example_content = example.contents
print(example_content)

[<a class="ga_w2gi_lp" data-gaact="Click_to_CityPage" data-galoc="Homedale - ID" dta-linktrack="City index page - Homedale" href="https://www.familydollar.com/locations/id/homedale/">Homedale</a>]


In [25]:
#Use .attr para encontrar quais atributos estão presentes no conteúdo deste objeto em forma de dicionário
#Nota: .contents geralmente retorna uma lista de exatamente um item; portanto, a primeira etapa é indexar esse item usando a notação de colchete.

example_content = example.contents[0]
example_content.attrs

{'class': ['ga_w2gi_lp'],
 'data-galoc': 'Homedale - ID',
 'data-gaact': 'Click_to_CityPage',
 'dta-linktrack': 'City index page - Homedale',
 'href': 'https://www.familydollar.com/locations/id/homedale/'}

In [26]:
#Agora podemos ver que "href" é um atributo e podemos extrair como um item de dicionário.

example_href = example_content['href']
print(example_href)

https://www.familydollar.com/locations/id/homedale/


**Montando o nosso web scraper**


In [27]:
#Abaixo temos a versão limpa do que descobrimos acima.

city_hrefs = [] # iniciando com uma lista vazia

for i in dollar_tree_list:#em cada item da lista
    cont = i.contents[0]#o conteudo do resultset é extraido
    href = cont['href']#retirado o link
    city_hrefs.append(href)# e adicionado a lista criada


In [29]:
#Checando os resultados
for i in city_hrefs[:3]:
  print(i)

https://www.familydollar.com/locations/id/aberdeen/
https://www.familydollar.com/locations/id/american-falls/
https://www.familydollar.com/locations/id/arco/


- Ainda não tenho informações de endereço. Agora, o URL de cada cidade precisa ser obtido para termos essas informações. Então, reiniciamos o processo, usando um exemplo único e representativo.
- As informações de endereço estão aninhadas em type = "application / ld + json". Felizmente, soup.find_all () também permite pesquisar no tipo.


In [35]:
page2 = requests.get(city_hrefs[2]) # Exemplo representativo
soup2 = BeautifulSoup(page2.text, 'html.parser')

In [37]:
arco = soup2.find_all(type="application/ld+json")
print(arco)

[<script type="application/ld+json">
    {
      "@context": "https://schema.org",
      "@type": "BreadcrumbList",
      "itemListElement": [{
        "@type": "ListItem",
        "position": 1,
        "item": {
          "@id": "https://www.familydollar.com/locations/",
          "name": "Index"
        }
      },{
        "@type": "ListItem",
        "position": 2,
        "item": {
          "@id": "https://www.familydollar.com/locations/id/",
          "name": "ID"
        }
      }]
    }
</script>, <script type="application/ld+json">
	{
	  "@context":"https://schema.org",
	  "@type":"Schema Business Type",
	  "name": "Family Dollar #9143",
	  "address":{
	    "@type":"PostalAddress",
	    "streetAddress":"157 W Grand Avenue",
	    "addressLocality":"Arco",
	    "addressRegion":"ID",
	    "postalCode":"83213",
	    "addressCountry":"US"
	  },
	  "containedIn":"",  
	  "branchOf": {
	    "name":"Family Dollar",
	    "url": "https://www.familydollar.com/"
	  },
	  "url":"https://w

- Agora conseguimos o endereço que está no segundo membro da lista!
- Extraí o conteúdo (do segundo item da lista) usando .contents (essa é uma boa ação padrão depois de filtrar a sopa). Novamente, como a saída do conteúdo é uma lista de um, indexei esse item da lista:

In [38]:
arco_contents = arco[1].contents[0]
arco_contents

'\n\t{\n\t  "@context":"https://schema.org",\n\t  "@type":"Schema Business Type",\n\t  "name": "Family Dollar #9143",\n\t  "address":{\n\t    "@type":"PostalAddress",\n\t    "streetAddress":"157 W Grand Avenue",\n\t    "addressLocality":"Arco",\n\t    "addressRegion":"ID",\n\t    "postalCode":"83213",\n\t    "addressCountry":"US"\n\t  },\n\t  "containedIn":"",  \n\t  "branchOf": {\n\t    "name":"Family Dollar",\n\t    "url": "https://www.familydollar.com/"\n\t  },\n\t  "url":"https://www.familydollar.com/locations/id/arco/29143/",\n\t  "telephone":"208-881-5738",\n\t  "image": "//hosted.where2getit.com/familydollarstore/images/storefront.png"\n\t}\t\t\t\n\t'

In [39]:
print(arco_contents)


	{
	  "@context":"https://schema.org",
	  "@type":"Schema Business Type",
	  "name": "Family Dollar #9143",
	  "address":{
	    "@type":"PostalAddress",
	    "streetAddress":"157 W Grand Avenue",
	    "addressLocality":"Arco",
	    "addressRegion":"ID",
	    "postalCode":"83213",
	    "addressCountry":"US"
	  },
	  "containedIn":"",  
	  "branchOf": {
	    "name":"Family Dollar",
	    "url": "https://www.familydollar.com/"
	  },
	  "url":"https://www.familydollar.com/locations/id/arco/29143/",
	  "telephone":"208-881-5738",
	  "image": "//hosted.where2getit.com/familydollarstore/images/storefront.png"
	}			
	


- O formato apresentado aqui é consistente com o formato JSON.
- Um objeto JSON pode atuar como um dicionário com dicionários aninhados dentro.
- Embora isso pareça estruturalmente com um objeto JSON, ainda é um objeto bs4 e precisa de uma conversão programática formal em JSON para ser acessada como um objeto JSON:


In [40]:
#Convertendo para o formato JSON

arco_json =  json.loads(arco_contents)

In [43]:
type(arco_json)

dict

In [42]:
print(arco_json)

{'@context': 'https://schema.org', '@type': 'Schema Business Type', 'name': 'Family Dollar #9143', 'address': {'@type': 'PostalAddress', 'streetAddress': '157 W Grand Avenue', 'addressLocality': 'Arco', 'addressRegion': 'ID', 'postalCode': '83213', 'addressCountry': 'US'}, 'containedIn': '', 'branchOf': {'name': 'Family Dollar', 'url': 'https://www.familydollar.com/'}, 'url': 'https://www.familydollar.com/locations/id/arco/29143/', 'telephone': '208-881-5738', 'image': '//hosted.where2getit.com/familydollarstore/images/storefront.png'}


In [46]:
#Nesse conteúdo, há uma chave chamada endereço que possui as informações de endereço desejadas no dicionário aninhado menor.

arco_address = arco_json['address']
arco_address

{'@type': 'PostalAddress',
 'streetAddress': '157 W Grand Avenue',
 'addressLocality': 'Arco',
 'addressRegion': 'ID',
 'postalCode': '83213',
 'addressCountry': 'US'}

- Agora eu posso percorrer os URLs da loja de lista em Idaho Falls:

In [47]:
locs_dict = [] # inicializa em uma lista vazia

for link in city_hrefs:
  locpage = requests.get(link)   # solicita infromação da pagina
  locsoup = BeautifulSoup(locpage.text, 'html.parser') 
      # analisa o conteúdo da pagina
  locinfo = locsoup.find_all(type="application/ld+json") 
      # extrai elemento expecifico onde contem o endereço
  loccont = locinfo[1].contents[0]  
      # pega o elemento em bs4
  locjson = json.loads(loccont)  # converte em json
  locaddr = locjson['address'] # pega o endereço
  locs_dict.append(locaddr)#adiciona a lista

- Temos muitos dados em um dicionário, mas temos algumas informações adicionais que tornarão a reutilização de nossos dados mais complexa do que precisa.
- Para executar algumas etapas finais da organização de dados, convertemos para um data frame do pandas, descartamos as colunas desnecessárias ("@type" e "country") e verificamos as cinco principais linhas para garantir que tudo fique bem.

In [54]:
locs_df = df.from_records(locs_dict)#tranforma em uma data frame
locs_df.drop(['@type', 'addressCountry', 'addressRegion'], axis = 1, inplace = True)#retira as variáveis '@type', 'addressCountry'.



In [55]:
locs_df.head(50)#Visualiza o data frame

,streetAddress,addressLocality,postalCode
0,111 N Main Street,Aberdeen,83210
1,253 Harrison St,American Falls,83211
2,157 W Grand Avenue,Arco,83213
3,177 Main Street,Ashton,83420
4,747 N. Main St.,Bellevue,83313
5,67 Sw Main St,Blackfoot,83221
6,2901 W State St.,Boise,83702
7,415 Broadway Ave S,Buhl,83316
8,1408 Overland Ave,Burley,83318
9,2402 E Linden St,Caldwell,83605


In [56]:
#Agora podemos salvar os nossos resultados.

df.to_csv(locs_df, "family_dollar_ID_locations.csv", sep = ",", index = False)

In [57]:
#Olhar também o Selenium